In [ ]:
import itertools
import math
import re
from pathlib import Path

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

plt.rcParams["figure.figsize"] = (16, 6)
# plt.rcParams["figure.autolayout"] = True # equivalent to .tight_layout(); breaks some legends
plt.rcParams["figure.constrained_layout.use"] = True  # Experimental

sns.set_style("whitegrid")
sns.set_context("notebook")

In [ ]:
def save_fig(fig, name, parent_dir="../figures/paper/", exts=["png", "pdf"]):

    p = Path(parent_dir)
    p.mkdir(exist_ok=True)

    for ext in exts:
        fn = p / f"{name}.{ext}"
        fig.savefig(fn)

    fig.show()

In [ ]:
# Take a list of labels and rename the ESCs from that list
# to more prettier (and shorter) names
def prettiefy_esc_names(labels):

    prettier_names = {
        "hvdc": "HVDC",
        "pipeline-h2": "H2 pipeline",
        "shipping-lh2": "H2 (l) ship",
        "shipping-lohc": "LOHC ship",
        "pipeline-ch4": "CH4 pipeline",
        "shipping-lch4": "CH4 (l) ship",
        "shipping-meoh": "MeOH ship",
        "shipping-lnh3": "NH3 (l) ship",
        "shipping-lnh3-to-h2": "NH3 (l) ship to H2 (g)",
        "shipping-ftfuel": "FTF ship",
    }

    if isinstance(labels, str):
        return prettier_names[labels]
    elif isinstance(labels, list):
        return [prettier_names[l] for l in labels]

In [ ]:
fn = "../results/default/results.csv"
fn = Path(fn)

df = pd.read_csv(fn, sep=";")  # , index_col=[0,1,2,3,4,5,6])
df = df.drop(columns=["scenario"])
df = df.set_index([c for c in df.columns if c != "value"], verify_integrity=True)

esc_markers = {
    "hvdc": "X",
    "pipeline-h2": "+",
    "shipping-lh2": "^",
    "shipping-lohc": "d",
    "pipeline-ch4": "*",
    "shipping-lch4": "v",
    "shipping-meoh": ">",
    "shipping-lnh3": "<",
    "shipping-ftfuel": "p",
}

cmap = matplotlib.cm.get_cmap("tab10")
exp_colors = {
    "AR": cmap(1),
    "AU": cmap(0),
    #'DE':cmap(7), # grey-ish
    "DE": (0, 0, 0, 1),  # black-ish
    "DK": cmap(6),
    "EG": cmap(3),
    "ES": cmap(2),
    "MA": cmap(4),
    "SA": cmap(5),
}

df = df.reorder_levels(
    ["year", "wacc", "importer", "category", "subcategory", "exporter", "esc"]
)

plt.figure(figsize=(10, 10))
ax = plt.gca()

tdf = df.loc[2030, "homogeneous", "DE"]

for exp, exp_c in exp_colors.items():
    for esc, esc_m in esc_markers.items():
        x = tdf.loc["general", "Cost per MWh delivered"].loc[exp, esc]["value"]
        y = tdf.loc["general", "Energy surplus factor"].loc[exp, esc]["value"]

        ax.scatter(x, y, marker=esc_m, color=exp_c, s=50)

ax.set_ylabel("Energy surplus factor")
ax.set_xlabel("EUR per MWh delivered")

# Construct legend
legend_esc = [
    matplotlib.lines.Line2D(
        [], [], color="None", linestyle="None", markersize=0, label="ESC", marker=None
    )
]
legend_esc += [
    matplotlib.lines.Line2D(
        [],
        [],
        color="white",
        linestyle="None",
        markersize=10,
        markeredgecolor="black",
        label=prettiefy_esc_names(n),
        marker=m,
    )
    for n, m in esc_markers.items()
]

legend_exp = [
    matplotlib.lines.Line2D(
        [],
        [],
        color="None",
        linestyle="None",
        markersize=0,
        label="Exporter",
        marker=None,
    )
]
legend_exp += [
    matplotlib.lines.Line2D(
        [], [], linestyle="None", markersize=10, marker=".", label=n, color=c
    )
    for n, c in exp_colors.items()
]

ax.legend(handles=legend_esc + legend_exp, ncol=2, loc="lower right")

# Minor gridlines
ax.minorticks_on()
ax.grid(which="minor")

# save figure
save_fig(plt.gcf(), "esf_vs_LCoEnergy")

In [ ]:
plt.figure(figsize=(10, 10))
ax = plt.gca()

tdf = df.loc[2030, "homogeneous", "DE"]

for exp, exp_c in exp_colors.items():
    for esc, esc_m in esc_markers.items():
        x = (
            tdf.loc["RES", "Curtailed electricity"].loc[exp, esc]["value"]
            / tdf.loc["RES", "Total produced electricity"].loc[exp, esc]["value"]
            * 100.0
        )
        y = tdf.loc["general", "Energy surplus factor"].loc[exp, esc]["value"]

        ax.scatter(x, y, marker=esc_m, color=exp_c, s=50)

ax.set_ylabel("Energy surplus factor")
ax.set_xlabel("Curtailed electricity [%]")

# Construct legend
legend_esc = [
    matplotlib.lines.Line2D(
        [], [], color="None", linestyle="None", markersize=0, label="ESC", marker=None
    )
]
legend_esc += [
    matplotlib.lines.Line2D(
        [],
        [],
        color="white",
        linestyle="None",
        markersize=10,
        markeredgecolor="black",
        label=prettiefy_esc_names(n),
        marker=m,
    )
    for n, m in esc_markers.items()
]

legend_exp = [
    matplotlib.lines.Line2D(
        [],
        [],
        color="None",
        linestyle="None",
        markersize=0,
        label="Exporter",
        marker=None,
    )
]
legend_exp += [
    matplotlib.lines.Line2D(
        [], [], linestyle="None", markersize=10, marker=".", label=n, color=c
    )
    for n, c in exp_colors.items()
]

ax.legend(handles=legend_esc + legend_exp, ncol=2, loc="lower right")

# Minor gridlines
ax.minorticks_on()
ax.grid(which="minor")

# save figure
save_fig(plt.gcf(), "esf_vs_curtailment")

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=(12, 6))

ax = ax1

tdf = df.loc[2030, "homogeneous", "DE"]

for exp, exp_c in exp_colors.items():
    for esc, esc_m in esc_markers.items():
        x = tdf.loc["general", "Cost per MWh delivered"].loc[exp, esc]["value"]
        y = tdf.loc["general", "Energy surplus factor"].loc[exp, esc]["value"]

        ax.scatter(x, y, marker=esc_m, color=exp_c, s=50)

ax.set_ylabel("Energy surplus factor")
ax.set_xlabel("EUR per MWh delivered")

# Construct legend
legend_esc = [
    matplotlib.lines.Line2D(
        [], [], color="None", linestyle="None", markersize=0, label="ESC", marker=None
    )
]
legend_esc += [
    matplotlib.lines.Line2D(
        [],
        [],
        color="white",
        linestyle="None",
        markersize=10,
        markeredgecolor="black",
        label=prettiefy_esc_names(n),
        marker=m,
    )
    for n, m in esc_markers.items()
]

legend_exp = [
    matplotlib.lines.Line2D(
        [],
        [],
        color="None",
        linestyle="None",
        markersize=0,
        label="Exporter",
        marker=None,
    )
]
legend_exp += [
    matplotlib.lines.Line2D(
        [], [], linestyle="None", markersize=10, marker=".", label=n, color=c
    )
    for n, c in exp_colors.items()
]

ax.legend(handles=legend_esc + legend_exp, ncol=2, loc="upper left", framealpha=0.2)

ax = ax2

tdf = df.loc[2030, "homogeneous", "DE"]

for exp, exp_c in exp_colors.items():
    for esc, esc_m in esc_markers.items():
        x = (
            tdf.loc["RES", "Curtailed electricity"].loc[exp, esc]["value"]
            / tdf.loc["RES", "Total produced electricity"].loc[exp, esc]["value"]
            * 100.0
        )
        y = tdf.loc["general", "Energy surplus factor"].loc[exp, esc]["value"]

        ax.scatter(x, y, marker=esc_m, color=exp_c, s=50)

# ax.set_ylabel('Energy surplus factor')
ax.set_xlabel("Curtailed electricity [%]")

# save figure
save_fig(plt.gcf(), "esf_vs_LCoEnergy-and-curtailment")

In [ ]:
## Supply curves (electricity)
# Use LCoEs from the same ESC for all (LCoElectricity does not depend on ESC, only on exporter)
fn = "../resources/networks_supplied/default/2030/hvdc/{exp}-DE/lcoes.csv"

demand_fn = "../resources/demand_annual_TRACES_2013.csv"
demand_df = pd.read_csv(demand_fn, sep=";", index_col="region")
demand_df = demand_df.rename(columns={"demand [GWh]": "demand [TWh]"}) / 1e3

from mpl_toolkits.axes_grid.inset_locator import (InsetPosition, inset_axes,
                                                  mark_inset)

fig = plt.figure(figsize=(8, 6))
ax = plt.gca()

## Create a set of inset Axes: these should fill the bounding box allocated to
# them.
ax2 = plt.axes([0, 0, 1, 1])
# Manually set the position and relative size of the inset axes within ax1
ip = InsetPosition(ax1, [0.3, 0.65, 0.6, 0.3])
ax2.set_axes_locator(ip)
ax.add_patch(
    plt.Rectangle(
        (0.02, 0.558), 0.420, 0.418, ls="-", ec="c", fc="white", transform=ax.transAxes
    )
)

for exp, exp_c in sorted(exp_colors.items()):
    df = pd.read_csv(fn.format(exp=exp))
    df["cumulative generation"] /= 1e6  # in TWh

    # Include demand as black dotted line
    demand = demand_df.loc[exp, "demand [TWh]"]
    df_lower = df[df["cumulative generation"] <= demand]
    df_upper = df[df["cumulative generation"] > demand]
    if not df_lower.empty:
        # Interpolate lcoe/ ylocation
        y_demand = np.interp(
            [demand],
            [
                df_lower.iloc[-1]["cumulative generation"],
                df_upper.iloc[0]["cumulative generation"],
            ],
            [df_lower.iloc[-1]["lcoe"], df_upper.iloc[0]["lcoe"]],
        )
        df_lower = df_lower.append(
            {"lcoe": y_demand[0], "cumulative generation": demand}, ignore_index=True
        ).sort_values("cumulative generation")
        df_upper = df_upper.append(
            {"lcoe": y_demand[0], "cumulative generation": demand}, ignore_index=True
        ).sort_values("cumulative generation")

        ax.plot(
            df_lower["cumulative generation"], df_lower["lcoe"], "--", ms=2, color="k"
        )
        ax2.plot(
            df_lower["cumulative generation"], df_lower["lcoe"], "--", ms=5, color="k"
        )

    ax.plot(
        df_upper["cumulative generation"],
        df_upper["lcoe"],
        "-o",
        ms=2,
        color=exp_c,
        label=exp,
    )
    ax2.plot(
        df_upper["cumulative generation"],
        df_upper["lcoe"],
        "-",
        ms=5,
        color=exp_c,
        label=exp,
    )

# ax.legend(ncol=len(exp_colors), loc='lower center')
ax.legend()
ax.set_xlim(1, 2.5e3)
ax.set_ylim(0, 125)
ax.set_ylabel("Marginal cost of electricity [EUR/MWh]")
ax.set_xlabel("Annual electricity potential [TWh]")

ax2.set_ylim(0, 200)
ax2.set_xlabel("[TWh]")
ax2.set_ylabel("[EUR/MWh]")

# save figure
save_fig(plt.gcf(), "supply-curves")

In [ ]:
## Share by technology of electricity
# Use LCoEs from the same ESC for all (LCoElectricity does not depend on ESC, only on exporter)
fn = "../resources/networks_supplied/default/2030/hvdc/{exp}-DE/lcoes.csv"

fig, axes = plt.subplots(3, 3, figsize=(10, 10))
axes_iter = itertools.cycle(axes.flatten())

for exp, exp_c in sorted(exp_colors.items()):
    df = pd.read_csv(fn.format(exp=exp))
    ax = next(axes_iter)
    df.groupby(df["technology"].str.replace("A|B", "")).sum()["annual generation"].plot(
        kind="pie",
        ax=ax,
        autopct="%1.0f%%",
        pctdistance=0.7,
        labeldistance=1.2,
        title=f"Annual generation potential for {exp}",
        ylabel="",
    )

ax = next(axes_iter)
ax.set_visible(False)

# save figure
save_fig(plt.gcf(), "res-annual-generation-potential-share")

In [ ]:
## Share by technology of electricity
# Use LCoEs from the same ESC for all (LCoElectricity does not depend on ESC, only on exporter)
fn = "../resources/networks_supplied/default/2030/hvdc/{exp}-DE/lcoes.csv"

fig, axes = plt.subplots(3, 3, figsize=(10, 10))
axes_iter = itertools.cycle(axes.flatten())

for exp, exp_c in sorted(exp_colors.items()):
    df = pd.read_csv(fn.format(exp=exp))
    ax = next(axes_iter)
    df.groupby(df["technology"].str.replace("A|B", "")).sum()["capacity"].plot(
        kind="pie",
        ax=ax,
        autopct="%1.0f%%",
        pctdistance=0.7,
        labeldistance=1.2,
        title=f"{exp}",
        ylabel="",
    )

ax = next(axes_iter)
ax.set_visible(False)

fig.suptitle("Installable capacity potentials")

# save figure
save_fig(plt.gcf(), "res-potentials")

In [ ]:
## Share by technology of installed capacity (installed in the model ESC HVDC, 2030, 10% homogeneous WACC)
fn = "../results/default/results.csv"
fn = Path(fn)

df = pd.read_csv(fn, sep=";")  # , index_col=[0,1,2,3,4,5,6])
df = df.drop(columns=["scenario"])

df = df.set_index([c for c in df.columns if c != "value"], verify_integrity=True)

subcategories = [
    "Total installed windonshore capacity",
    "Total installed windoffshore capacity",
    "Total installed pvplant capacity",
]

df = df.loc[2030, "homogeneous", :, :, "DE", "RES", subcategories]

df = df.reset_index().pivot(
    index=["esc", "exporter"], columns=["subcategory"], values="value"
)

# Rename to nicer column names / labels
df = df.rename(
    columns=lambda x: re.match("Total installed (\S+) capacity", x).groups()[0]
)

df.columns.name = ""

fig, axes = plt.subplots(3, 3, figsize=(10, 10))
axes_iter = itertools.cycle(axes.flatten())

for esc, esc_m in esc_markers.items():
    ax = next(axes_iter)
    df.loc[esc].plot(
        kind="bar", stacked=True, ax=ax, title=f"{prettiefy_esc_names(esc)}"
    )
    legend = ax.legend()
    legend.set_visible(False)
    ax.ticklabel_format(
        axis="y", style="sci", scilimits=(0, 0)
    )  # scientific notation for yaxis

legend.set_visible(True)

fig.suptitle("Installed capacities [MW]")
# save figure
save_fig(plt.gcf(), "res-installed-capacity")

In [ ]:
## Share by technology of generated electricity (in the model ESC HVDC, 2030, 10% homogeneous WACC)
fn = "../results/default/results.csv"
fn = Path(fn)

df = pd.read_csv(fn, sep=";")  # , index_col=[0,1,2,3,4,5,6])
df = df.drop(columns=["scenario"])
df = df.set_index([c for c in df.columns if c != "value"], verify_integrity=True)

subcategories = [
    "Total produced electricity from windonshore",
    "Total produced electricity from windoffshore",
    "Total produced electricity from pvplant",
]

df = df.loc[2030, "homogeneous", :, :, "DE", "RES", subcategories]

df = df.reset_index().pivot(
    index=["esc", "exporter"], columns=["subcategory"], values="value"
)

# Rename to nicer column names / labels
df = df.rename(
    columns=lambda x: re.match("Total produced electricity from (\S+)", x).groups()[0]
)

df.columns.name = ""

fig, axes = plt.subplots(3, 3, figsize=(10, 10))
axes_iter = itertools.cycle(axes.flatten())

for esc, esc_m in esc_markers.items():
    ax = next(axes_iter)
    df.loc[esc].plot(
        kind="bar", stacked=True, ax=ax, title=f"{prettiefy_esc_names(esc)}"
    )
    legend = ax.legend(ncol=1)
    legend.set_visible(False)

legend.set_visible(True)
fig.suptitle("Generated electricity [MWh]")
# save figure
save_fig(plt.gcf(), "res-generated-electricity")

In [ ]:
## Costs of energy by ESC
fn = "../results/default/results.csv"
fn = Path(fn)

df = pd.read_csv(fn, sep=";")  # , index_col=[0,1,2,3,4,5,6])
df = df.drop(columns=["scenario"])
df = df.set_index(
    ["year", "wacc", "importer", "category", "subcategory", "esc", "exporter"]
)

df = df.loc[2030, "homogeneous", "DE", "general", "Cost per MWh delivered"]

xs = np.arange((len(esc_markers)))  # label locations
xxs = np.arange(len(exp_colors))  # sublocations within each esc
width = 0.1  # bar widths

fig = plt.figure(figsize=(12, 4))
ax = plt.gca()

for (esc, x) in zip(esc_markers.keys(), xs):
    for ((idx, row), xx) in zip(df.loc[esc].sort_values("value").iterrows(), xxs):
        ax.bar(
            x + xx * width - (xxs.shape[0] - 1) * width / 2,
            row["value"],
            width,
            color=exp_colors[idx],
        )

# Construct legend
legend_exp = [
    matplotlib.lines.Line2D(
        [],
        [],
        color="None",
        linestyle="None",
        markersize=0,
        label="Exporter",
        marker=None,
    )
]
legend_exp += [
    matplotlib.lines.Line2D(
        [], [], linestyle="None", markersize=15, marker=".", label=n, color=c
    )
    for n, c in exp_colors.items()
]

ax.legend(handles=legend_exp, ncol=9, loc="upper left")

ax.set_xticks(xs)
ax.set_xticklabels(prettiefy_esc_names(list(esc_markers.keys())))

ax.set_ylabel("Cost per energy delivered [EUR/MWh]")

# save figure
save_fig(plt.gcf(), "LCoEnergy_per_ESC-EXP")

In [ ]:
## Cost development across years
fn = "../results/default/results.csv"
fn = Path(fn)

df = pd.read_csv(fn, sep=";")  # , index_col=[0,1,2,3,4,5,6])
df = df.drop(columns=["scenario"])
df = df.set_index(
    ["wacc", "importer", "category", "subcategory", "esc", "year", "exporter"]
)

df = df.loc["homogeneous", "DE", "general", "Cost per MWh delivered"]

years = [2030, 2040, 2050]

xs = np.arange((len(esc_markers)))  # label locations
xxs = np.arange(len(years))  # sublocations within each esc
width = 0.2  # bar widths

fig = plt.figure(figsize=(10, 6))
ax = plt.gca()

for (esc, x) in zip(esc_markers.keys(), xs):
    for (year, xx) in zip(years, xxs):
        for idx, row in df.loc[esc, year].iterrows():
            # Single value dots
            ax.scatter(
                x + xx * width - (xxs.shape[0] - 1) * width / 2,
                row["value"],
                color=exp_colors[idx],
                marker=".",
                s=120,
                zorder=2,
                ec="k",
            )

        # Connecting bar
        ax.plot(
            [x + xx * width - (xxs.shape[0] - 1) * width / 2] * 2,
            [df.loc[esc, year].min(), df.loc[esc, year].max()],
            zorder=1,
            color="gray",
            lw=3,
            alpha=0.5,
        )
        # Year label
        ax.text(
            x + xx * width - (xxs.shape[0] - 1) * width / 2,
            50,
            year,
            rotation=90,
            va="top",
        )

ax.set_ylim(
    0,
)
ax.set_xticks(xs)
ax.set_xticklabels(prettiefy_esc_names(list(esc_markers.keys())))

ax.set_ylabel("Cost per energy delivered [EUR/MWh]")
ax.set_xlabel("ESC")

# Created the same legend above. Reusing for simplicity
ax.legend(handles=legend_exp, ncol=9, loc="upper left")

# No grid lines, add manually later
sns.set_style("white")

# Minor gridlines
ax.minorticks_on()
ax.grid(which="minor", axis="y", color="gray", alpha=0.3)
ax.grid(which="major", axis="y")

# save figure
save_fig(plt.gcf(), f'LCoEnergy_per_ESC_{"-".join([str(y) for y in years])}')

In [ ]:
## Cost development across years
fn = "../results/lowhomogeneous/results.csv"
fn = Path(fn)

df = pd.read_csv(fn, sep=";")  # , index_col=[0,1,2,3,4,5,6])
df = df.drop(columns=["scenario"])
df = df.set_index(
    ["wacc", "importer", "category", "subcategory", "esc", "year", "exporter"]
)

df = df.loc["lowhomogeneous", "DE", "general", "Cost per MWh delivered"]

years = [2030, 2040, 2050]

xs = np.arange((len(esc_markers)))  # label locations
xxs = np.arange(len(years))  # sublocations within each esc
width = 0.2  # bar widths

fig = plt.figure(figsize=(10, 6))
ax = plt.gca()

for (esc, x) in zip(esc_markers.keys(), xs):
    for (year, xx) in zip(years, xxs):
        for idx, row in df.loc[esc, year].iterrows():
            # Individual dots
            ax.scatter(
                x + xx * width - (xxs.shape[0] - 1) * width / 2,
                row["value"],
                color=exp_colors[idx],
                marker=".",
                s=120,
                zorder=2,
                ec="k",
            )
        # Connecting bar
        ax.plot(
            [x + xx * width - (xxs.shape[0] - 1) * width / 2] * 2,
            [df.loc[esc, year].min(), df.loc[esc, year].max()],
            zorder=1,
            color="gray",
            lw=3,
            alpha=0.5,
        )
        # Year label
        ax.text(
            x + xx * width - (xxs.shape[0] - 1) * width / 2,
            35,
            year,
            rotation="vertical",
            va="top",
        )

ax.set_ylim(
    0,
)
ax.set_xticks(xs)
ax.set_xticklabels(prettiefy_esc_names(list(esc_markers.keys())))

ax.set_ylabel("Cost per energy delivered [EUR/MWh]")
ax.set_xlabel("ESC")

# Created the same legend above. Reusing for simplicity
ax.legend(handles=legend_exp, ncol=9, loc="upper left")

# No grid lines, add manually later
sns.set_style("white")

# Minor gridlines
ax.minorticks_on()
ax.grid(which="minor", axis="y", color="gray", alpha=0.3)
ax.grid(which="major", axis="y")

# save figure
save_fig(
    plt.gcf(), f'LCoEnergy_per_ESC_{"-".join([str(y) for y in years])}_lowhomogeneous'
)

In [ ]:
## Cost development per t H2 across the years
fn = "../results/default/results.csv"
fn = Path(fn)

df = pd.read_csv(fn, sep=";")  # , index_col=[0,1,2,3,4,5,6])
df = df.drop(columns=["scenario"])
df = df.set_index(
    ["wacc", "importer", "category", "subcategory", "esc", "year", "exporter"]
)

df = df.loc["homogeneous", "DE", "general", "Cost per MWh delivered"]

# Conversion factor to convert MWh (for chemical in each ESC) to 1 t_H2 content
# Derived from LHV/(H2-content by weight in p.u.)
# i.e. MWh_x/t_H2
conversion_factors = {
    "hvdc": 33.3333 / 1.0,
    "pipeline-h2": 33.3333 / 1.0,
    "shipping-lh2": 33.3333 / 1.0,
    "shipping-lohc": 33.3333 / 1.0,
    #'pipeline-ch4':  (50./3.6)/0.25,
    "pipeline-ch4-to-h2": 33.3333 / 1.0,
    #'shipping-lch4': (50./3.6)/0.25,
    #'shipping-meoh': (19.3/3.6)/0.125,
    #'shipping-lnh3': (18.6/3.6)/(3/17),
    "shipping-lch4-to-h2": 33.3333 / 1.0,
    "shipping-meoh-to-h2": 33.3333 / 1.0,
    "shipping-lnh3-to-h2": 33.3333 / 1.0,
}

df = df.loc[conversion_factors.keys()]

# Rescale costs per MWh_th to Costs per kg_H2 content
df["value"] = (
    df.apply(lambda x: conversion_factors[x.name[0]] * x["value"], axis=1) / 1e3
)
df.name = "Cost per kg_H2 delivered [EUR/kg_H2]"

years = [2030, 2040, 2050]

xs = np.arange((len(conversion_factors)))  # label locations
xxs = np.arange(len(years))  # sublocations within each esc
width = 0.2  # bar widths

fig = plt.figure(figsize=(10, 6))
ax = plt.gca()

# Plot each vertical year line and each scatter point separately
for (esc, x) in zip(conversion_factors.keys(), xs):
    for (year, xx) in zip(years, xxs):
        for idx, row in df.loc[esc, year].iterrows():
            # each scatter/data point. Marked by color of exporter
            ax.scatter(
                x + xx * width - (xxs.shape[0] - 1) * width / 2,
                row["value"],
                color=exp_colors[idx],
                marker=".",
                s=120,
                zorder=2,
                ec="k",
            )

        # Vertical line per year
        ax.plot(
            [x + xx * width - (xxs.shape[0] - 1) * width / 2] * 2,
            [df.loc[esc, year].min(), df.loc[esc, year].max()],
            zorder=1,
            color="gray",
            lw=3,
            alpha=0.5,
        )
        # Year indicator
        ax.text(
            x + xx * width - (xxs.shape[0] - 1) * width / 2,
            0.1,
            year,
            rotation="vertical",
            va="bottom",
        )

ax.set_ylim(
    0,
)
ax.set_xticks(xs)
ax.set_xticklabels(
    prettiefy_esc_names([l.replace("-to-h2", "") for l in conversion_factors.keys()])
)

ax.set_ylabel("Cost per hydrogen delivered [EUR/kg-H2]")
ax.set_xlabel("ESC")

# Created the same legend above. Reusing for simplicity
ax.legend(handles=legend_exp, ncol=9, loc="upper left")

# No grid lines, add manually later
sns.set_style("white")

# Minor gridlines
ax.minorticks_on()
ax.grid(which="minor", axis="y", color="gray", alpha=0.3)
ax.grid(which="major", axis="y")

# save figure
save_fig(plt.gcf(), f'LCoH_per_ESC_{"-".join([str(y) for y in years])}')

In [ ]:
## Cost development per t H2 across the years
fn = "../results/lowhomogeneous/results.csv"
fn = Path(fn)

df = pd.read_csv(fn, sep=";")  # , index_col=[0,1,2,3,4,5,6])
df = df.drop(columns=["scenario"])
df = df.set_index(
    ["wacc", "importer", "category", "subcategory", "esc", "year", "exporter"]
)

df = df.loc["lowhomogeneous", "DE", "general", "Cost per MWh delivered"]

# Conversion factor to convert MWh (for chemical in each ESC) to 1 t_H2 content
# Derived from LHV/(H2-content by weight in p.u.)
# i.e. MWh_x/t_H2
conversion_factors = {
    "hvdc": 33.3333 / 1.0,
    "pipeline-h2": 33.3333 / 1.0,
    "shipping-lh2": 33.3333 / 1.0,
    "shipping-lohc": 33.3333 / 1.0,
    #'pipeline-ch4':  (50./3.6)/0.25,
    "pipeline-ch4-to-h2": 33.3333 / 1.0,
    #'shipping-lch4': (50./3.6)/0.25,
    #'shipping-meoh': (19.3/3.6)/0.125,
    #'shipping-lnh3': (18.6/3.6)/(3/17),
    "shipping-lch4-to-h2": 33.3333 / 1.0,
    "shipping-meoh-to-h2": 33.3333 / 1.0,
    "shipping-lnh3-to-h2": 33.3333 / 1.0,
}

df = df.loc[conversion_factors.keys()]

# Rescale costs per MWh_th to Costs per kg_H2 content
df["value"] = (
    df.apply(lambda x: conversion_factors[x.name[0]] * x["value"], axis=1) / 1e3
)
df.name = "Cost per kg_H2 delivered [EUR/kg_H2]"

years = [2030, 2040, 2050]

xs = np.arange((len(conversion_factors)))  # label locations
xxs = np.arange(len(years))  # sublocations within each esc
width = 0.2  # bar widths

fig = plt.figure(figsize=(10, 6))
ax = plt.gca()

# Plot each vertical year line and each scatter point separately
for (esc, x) in zip(conversion_factors.keys(), xs):
    for (year, xx) in zip(years, xxs):
        for idx, row in df.loc[esc, year].iterrows():
            # each scatter/data point. Marked by color of exporter
            ax.scatter(
                x + xx * width - (xxs.shape[0] - 1) * width / 2,
                row["value"],
                color=exp_colors[idx],
                marker=".",
                s=120,
                zorder=2,
                ec="k",
            )

        # Vertical line per year
        ax.plot(
            [x + xx * width - (xxs.shape[0] - 1) * width / 2] * 2,
            [df.loc[esc, year].min(), df.loc[esc, year].max()],
            zorder=1,
            color="gray",
            lw=3,
            alpha=0.5,
        )
        # Year indicator
        ax.text(
            x + xx * width - (xxs.shape[0] - 1) * width / 2,
            0.1,
            year,
            rotation="vertical",
            va="bottom",
        )

ax.set_ylim(
    0,
)
ax.set_xticks(xs)
ax.set_xticklabels(
    prettiefy_esc_names([l.replace("-to-h2", "") for l in conversion_factors.keys()])
)

ax.set_ylabel("Cost per hydrogen delivered [EUR/kg-H2]")
ax.set_xlabel("ESC")

# Created the same legend above. Reusing for simplicity
ax.legend(handles=legend_exp, ncol=9, loc="upper left")

# No grid lines, add manually later
sns.set_style("white")

# Minor gridlines
ax.minorticks_on()
ax.grid(which="minor", axis="y", color="gray", alpha=0.3)
ax.grid(which="major", axis="y")

# save figure
save_fig(plt.gcf(), f'LCoH_per_ESC_{"-".join([str(y) for y in years])}_lowhomogeneous')

In [ ]:
## Cost composition for selected ESCs, EXPs
fn = "../results/default/results.csv"
fn = Path(fn)

df = pd.read_csv(fn, sep=";")  # , index_col=[0,1,2,3,4,5,6])
df = df.drop(columns=["scenario"])
df = df.set_index(["year", "wacc", "importer", "category"])

# Used later, assume demand for all ESCs is identical
demand = df.query('subcategory=="Total demand"').iloc[0]["value"]

df = df.loc[2030, "homogeneous", "DE", "cost"].reset_index(drop=True)
selected_exps = ["AR", "DE", "ES"]
selected_escs = [
    "hvdc",
    "pipeline-h2",
    "shipping-lh2",
    "shipping-lohc",
    "pipeline-ch4",
    "shipping-lch4",
    "shipping-meoh",
    "shipping-ftfuel",
    "shipping-lnh3",
    "shipping-lnh3-to-h2",
]

df = df[(df["esc"].isin(selected_escs)) & df["exporter"].isin(selected_exps)]
df = df.pivot(index=["esc", "exporter"], columns="subcategory", values="value")

# Aggregate columns and nicer names
# Order here also directs the order in the plot later
columns_to_aggregate = {
    "PV | wind": "windonshore|windoffshore|pvplant",
    "water desalination & storage": "(seawater desalination|clean water tank storage)",
    "battery storage": "battery",
    "electrolysis": "electrolysis",
    #'heat pump':'industrial heat pump medium temperature',
    #'DAC':'direct air capture',
    "DAC incl. heat pump | LOHC chemical": (
        "(direct air capture)|(industrial heat pump medium temperature)|"
        "^((LOHC chemical)|(LOHC treatment)|(LOHC \(loaded\) (un)?loading)|(LOHC \(used\) (un)?loading))"
    ),
    "storage H2 (g) incl. compression": "(hydrogen demand buffer)|(hydrogen \(g\) demand buffer)|(hydrogen storage)|(H2 storage)",  # Storage + buffer + compressors
    "storage CO2 (l) incl. conversion": "CO2 (liquefaction|evaporation|storage tank)",
    "storage CH4 (g) incl. compression": "(CH4 storage incl. compression)|(CH4 storage)|(methane storage)|(methane \(g\) demand buffer)",
    #'storage hydrocarbon product':'(((FT fuel)|methanol) demand buffer)|(((FT fuel)|MeOH) storing)|(General liquid hydrocarbon storage)',
    #'storage LOHC':'LOHC (unloaded|loaded) DBT storage',
    #'storage CH4':'(CH4 storage)|(methane storage)|(methane \(g\) demand buffer)', # CH4 storage and compressor cost
    #'storage NH3':'NH3 ((evaporation|liquefaction)|(\(l\) storage tank incl. liquefaction))|ammonia \(g\) demand buffer',
    "storage H2(l) | LOHC | CH4(l) | NH3(l) | MeOH | FT fuel": (
        "(H2 \(l\) storage tank)|"
        "(FT fuel demand buffer)|"
        "(methanol demand buffer)|"
        "(FT fuel storing)|"
        "(MeOH storing)|"
        "(General liquid hydrocarbon storage)|"
        "(LOHC (unloaded|loaded) DBT storage)|"
        "(LNG storage tank)|(CH4 \(l\) storing and unstoring)|"
        "NH3 ((evaporation|liquefaction)|(\(l\) storage tank incl. liquefaction))|ammonia \(g\) demand buffer"
    ),
    #'Methanation':'methanation',
    #'Methanol synthesis': 'methanolisation',
    #'Ammonia synthesis':'(air separation unit|Haber-Bosch)',
    #'Fischer-Tropsch':'Fischer-Tropsch',
    "synthesis CH4 | NH3| MeOH | FT fuel": (
        "(methanation)|"
        "(methanolisation)|"
        "(air separation unit)|(Haber-Bosch)|"
        "(Fischer-Tropsch)"
    ),
    #'de-/hydrogenation LOHC': 'LOHC (de)?hydrogenation',
    #'Ammonia to Hydrogen cracking': 'Ammonia cracker',
    "H2(l) & CH4(l) liquefaction & evaporation | LOHC (de-)hydrogenation | NH3 cracking": (
        "(LOHC (de)?hydrogenation)|"
        "(Ammonia cracker)|"
        "H2 ((evaporation|liquefaction))|"
        "CH4 ((evaporation|liquefaction))"
    ),
    #'HVDC': 'HVDC',
    #'Pipeline H2 (g) / CH4 (g)':'(H2|CH4) \(g\)( submarine)? pipeline',
    #'Shipping':'(transport )?ship\s?(un)?(loading)?', # Match transport ship as well as (un-) loading operations
    "HVDC | Pipeline | Shipping": "((HVDC)|((H2|CH4) \(g\)( submarine)? pipeline)|((H2|CH4) \(g\) fill compressor station)|((transport )?ship\s?(un)?(loading)?))",
}

# Rename and aggregate
for c_new, c in columns_to_aggregate.items():
    tmp = df.filter(regex=c, axis=1)
    if tmp.empty:
        continue
    df = df.drop(tmp.columns, axis=1)
    df[c_new] = tmp.sum(axis=1)

# Colormap selection/creation and iterator
number_of_colors = 13
cmap = plt.get_cmap("tab20c_r").reversed()
colors = itertools.cycle([cmap(i) for i in np.arange(number_of_colors)])

fig = plt.figure(figsize=(14, 6))
ax = plt.gca()

# Copy df and sort according to order of selected_escs list (to show in same order in plot)
tdf = df.copy().reindex(selected_escs, level=0)

legend_handles = []

for v in list(columns_to_aggregate.keys())[::-1]:  # backwards iteration necessary

    # New Skip? -> Reverse order relevant, i.e. from bottom to top in above enter those to skip
    if any(
        [
            v.startswith(s)
            for s in [
                "bla",
                #'Pipeline', 'HVDC', # same as shipping
                #'DAC', # same as LOHC chemical
                #'storage CH4','storage LOHC','storage hydrocarbon product', # same as storage NH3
                #'Ammonia synthesis','Methanol synthesis','Methanation','de-/hydrogenation LOHC' # same as FT synth.
            ]
        ]
    ):
        # Do not use a new color to reduce the color variety
        # the meaning of the color is inferable from the ESC
        # The list to skip is determined by the backwards order of the dict 'columns_to_aggregate'
        print(f"skipping new color for {v}")
    else:
        # New color for new technology
        c = next(colors)

    # Plotting a stacked grouped bar plot is stupid
    # we plot the bars and then plot over them with the lower values
    # need to sum() manually in each iteration after removing a column in the previous iteration
    # then create dedicated dataframes
    # reindex to preserve order in plot
    (
        (
            tdf.sum(axis=1)
            .to_frame(name="sum")
            .reset_index()
            .pivot(index="esc", columns="exporter", values="sum")
            .reindex(selected_escs)
        )
        / demand
    ).plot(
        kind="bar", ax=ax, color=[c]
    )  # pandas needs - for whatever reason - an iterable containing a tuple for RGBA (instead of just 'c'))
    tdf = tdf.drop(columns=v)

    # Add appropriate legend entry for technology
    legend_handles += [
        matplotlib.lines.Line2D(
            [],
            [],
            linestyle="None",
            markersize=10,
            markeredgecolor="black",
            marker="s",
            label=v,
            color=c,
        )
    ]

# Header for legend
legend_handles = [
    matplotlib.lines.Line2D(
        [],
        [],
        color="None",
        linestyle="None",
        markersize=0,
        label="Component",
        marker=None,
    )
] + legend_handles
ax.legend(handles=legend_handles, loc="upper left")
# ax.legend(handles=legend_handles, bbox_to_anchor=(1.05, 1), loc='center left')

# axis labels
ax.set_ylabel("Cost per MWh chemical delivered [EUR/MWh]")
ax.set_xlabel("ESC and exporter")
plt.xticks(rotation=0)  # Remove automatic rotation from pandas

# Add labels for exporters
width = ax.patches[0].get_width()
for x in ax.get_xticks():
    for i, exp in enumerate(selected_exps):
        ax.text(x + width * (i - 1.5), -20, exp, size="small")

ax.set_xticklabels(
    ["\n" + prettiefy_esc_names(esc.get_text()) for esc in ax.get_xticklabels()]
)

sns.set_style("whitegrid")

# save figure
save_fig(plt.gcf(), f"cost-compositions_selected-ESC-EXP")

In [ ]:
## Results sensitivity analysis
# ES H2 pipeline 2030
esc = "pipeline-h2"

fns = {
    "reference": "../results/default/results.csv",
    "WACC": "../results/sensitivity-1/results.csv",  # Names are w/o addition for (-20%) and with "p" after the number for "+20%"
    "Import volume": "../results/sensitivity-2/results.csv",
    "Domestic demand": "../results/sensitivity-3/results.csv",
    "CAPEX RES": "../results/sensitivity-4/results.csv",
    "CAPEX battery": "../results/sensitivity-5/results.csv",
    "CAPEX electrolysis": "../results/sensitivity-6/results.csv",
    "CAPEX pipeline": "../results/sensitivity-8/results.csv",
}

# Load reference value
fn = fns.pop("reference")
df = pd.read_csv(fn, sep=";", index_col=[0, 1, 2, 3, 4, 5, 6])
reference_value = df.loc[
    2030, "homogeneous", esc, "ES", "DE", "general", "Cost per MWh delivered"
]["value"]

# Construct dataframe with sensitivity runs
l = []
for k, fn in fns.items():

    m = (
        pd.read_csv(fn, sep=";", index_col=["esc", "subcategory"])
        .loc[esc, "Cost per MWh delivered"]["value"]
        .item()
    )
    p = (
        pd.read_csv(
            fn.replace("/results.csv", "p/results.csv"),
            sep=";",
            index_col=["esc", "subcategory"],
        )
        .loc[esc, "Cost per MWh delivered"]["value"]
        .item()
    )

    l.append([k, m, p])

df = pd.DataFrame(l, columns=["parameter", "-20%", "+20%"])

# Tornado structure from wide to narrow
df = df.sort_values("-20%", ascending=False)

df = df.reset_index(drop=True)

# highest difference between reference and any synsitivity, rounded up to next 5er
max_diff = 5 * np.ceil(
    np.max([reference_value - df["-20%"].min(), df["+20%"].max() - reference_value]) / 5
)

fig, ax = plt.subplots(figsize=(8, 4.5))
ax.invert_yaxis()

h = 0.4

# central black line
ax.plot([reference_value, reference_value], [-1, len(df) + 1], color="k")

ax.set_yticks(list(df.index))
# ax.set_yticklabels([s+' -20%' for s in df['parameter'].to_list()])
ax.set_yticklabels([s for s in df["parameter"].to_list()])
ax.set_xlim([reference_value - max_diff, reference_value + max_diff])
ax.set_ylim(0 - h, len(df) - 1 + h)
ax.set_xlabel("Cost per MWh [EUR]")

# additional upper/right axes
axx = ax.twinx()
# axx.set_yticks(list(df.index))
axx.set_yticks(list())
# axx.set_yticklabels([' +20%' for s in df['parameter'].to_list()])
axx.set_ylim(ax.get_ylim())
axy = ax.twiny()
axy.set_xticks(np.linspace(0.9, 1.1, 5) * reference_value)
axy.set_xticklabels([f"{v:.0f}%" for v in np.linspace(90, 110, 5)])
axy.set_xlim(ax.get_xlim())

# Left/Right side indicator for +/- 20%
ax.text(
    0.05,
    0.5,
    "-20%",
    horizontalalignment="center",
    verticalalignment="center",
    rotation="vertical",
    transform=ax.transAxes,
)
axx.text(
    0.95,
    0.5,
    "+20%",
    horizontalalignment="center",
    verticalalignment="center",
    rotation="vertical",
    transform=ax.transAxes,
)

for idx, row in df.iterrows():
    w = reference_value - row["-20%"]
    ax.barh(idx, width=w, height=h, left=row["-20%"], fc="gray")
    ax.barh(
        y=idx,
        left=reference_value,
        width=row["+20%"] - reference_value,
        height=h,
        fc="gray",
    )
    ax.text(
        x=reference_value * 0.995,
        y=idx,
        s="{s:+.1f}%".format(s=(row["-20%"] - reference_value) / reference_value * 100),
        ha="right",
        va="center",
    )
    ax.text(
        x=reference_value * 1.005,
        y=idx,
        s="{s:+.1f}%".format(s=(row["+20%"] - reference_value) / reference_value * 100),
        ha="left",
        va="center",
    )

sns.set_style("white")

# save figure
save_fig(plt.gcf(), f"sensitivities_2030_{esc}_ES-DE")

In [ ]:
sns.set_style("white")


## Results sensitivity analysis
# ES H2 pipeline 2030
esc = "shipping-meoh"

fns = {
    "reference": "../results/default/results.csv",
    "WACC": "../results/sensitivity-1/results.csv",  # Names are w/o addition for (-20%) and with "p" after the number for "+20%"
    "Import volume": "../results/sensitivity-2/results.csv",
    "Domestic demand": "../results/sensitivity-3/results.csv",
    "CAPEX RES": "../results/sensitivity-4/results.csv",
    "CAPEX battery": "../results/sensitivity-5/results.csv",
    "CAPEX electrolysis": "../results/sensitivity-6/results.csv",
    "CAPEX MeOH synthesis": "../results/sensitivity-7/results.csv",
}

# Load reference value
fn = fns.pop("reference")
df = pd.read_csv(fn, sep=";", index_col=[0, 1, 2, 3, 4, 5, 6])
reference_value = df.loc[
    2030, "homogeneous", esc, "ES", "DE", "general", "Cost per MWh delivered"
]["value"]

# Construct dataframe with sensitivity runs
l = []
for k, fn in fns.items():

    m = (
        pd.read_csv(fn, sep=";", index_col=["esc", "subcategory"])
        .loc[esc, "Cost per MWh delivered"]["value"]
        .item()
    )
    p = (
        pd.read_csv(
            fn.replace("/results.csv", "p/results.csv"),
            sep=";",
            index_col=["esc", "subcategory"],
        )
        .loc[esc, "Cost per MWh delivered"]["value"]
        .item()
    )

    l.append([k, m, p])

df = pd.DataFrame(l, columns=["parameter", "-20%", "+20%"])

# Tornado structure from wide to narrow
df = df.sort_values("-20%", ascending=False)

df = df.reset_index(drop=True)

# highest difference between reference and any synsitivity, rounded up to next 5er
max_diff = 5 * np.ceil(
    np.max([reference_value - df["-20%"].min(), df["+20%"].max() - reference_value]) / 5
)

fig, ax = plt.subplots(figsize=(8, 4.5))
ax.invert_yaxis()

h = 0.4

# central black line
ax.plot([reference_value, reference_value], [-1, len(df) + 1], color="k")

ax.set_yticks(list(df.index))
# ax.set_yticklabels([s+' -20%' for s in df['parameter'].to_list()])
ax.set_yticklabels([s for s in df["parameter"].to_list()])
ax.set_xlim([reference_value - max_diff, reference_value + max_diff])
ax.set_ylim(0 - h, len(df) - 1 + h)
ax.set_xlabel("Cost per MWh [EUR]")

# additional upper/right axes
axx = ax.twinx()
# axx.set_yticks(list(df.index))
# axx.set_yticklabels([' +20%' for s in df['parameter'].to_list()])
axx.set_yticks([])
axx.set_ylim(ax.get_ylim())
axy = ax.twiny()
axy.set_xticks(np.linspace(0.9, 1.1, 5) * reference_value)
axy.set_xticklabels([f"{v:.0f}%" for v in np.linspace(90, 110, 5)])
axy.set_xlim(ax.get_xlim())

ax.text(
    0.05,
    0.5,
    "-20%",
    horizontalalignment="center",
    verticalalignment="center",
    rotation="vertical",
    transform=ax.transAxes,
)
axx.text(
    0.95,
    0.5,
    "+20%",
    horizontalalignment="center",
    verticalalignment="center",
    rotation="vertical",
    transform=ax.transAxes,
)

for idx, row in df.iterrows():
    w = reference_value - row["-20%"]
    ax.barh(idx, width=w, height=h, left=row["-20%"], fc="gray")
    ax.barh(
        y=idx,
        left=reference_value,
        width=row["+20%"] - reference_value,
        height=h,
        fc="gray",
    )
    ax.text(
        x=reference_value * 0.995,
        y=idx,
        s="{s:+.1f}%".format(s=(row["-20%"] - reference_value) / reference_value * 100),
        ha="right",
        va="center",
    )
    ax.text(
        x=reference_value * 1.005,
        y=idx,
        s="{s:+.1f}%".format(s=(row["+20%"] - reference_value) / reference_value * 100),
        ha="left",
        va="center",
    )

# save figure
save_fig(plt.gcf(), f"sensitivities_2030_{esc}_ES-DE")